In [ ]:
%load_ext autoreload
%autoreload 2
%store -r the_page

if 'the_page' not in locals():
    import pickle
    print("Loading default data...")
    the_page = pickle.load(open("data/the_page.p",'rb'))

from IPython.display import display, Markdown as md
display(md("---"))
display(md(f"# A. Insertions, Deletions, Reinsertions (Actions)"))
display(md(f"***Page: {the_page['title']}***"))

In [ ]:
from wikiwho_wrapper import WikiWho
import pandas as pd
import qgrid
# set the default max number of rows to 10 so the larger DataFrame we render don't take up to much space 
qgrid.set_grid_option('maxVisibleRows', 5)

wikiwho = WikiWho(lng='en')
agg_actions = wikiwho.dv.actions(the_page.page_id)

# define total columns
total_columns = ['total', 'total_surv_48h', 'total_persistent', 'total_stopword_count']

# add columns with the total actions
agg_actions = agg_actions.join(pd.DataFrame(
    agg_actions.loc[:,'adds':'adds_stopword_count'].values +\
    agg_actions.loc[:,'dels':'dels_stopword_count'].values +\
    agg_actions.loc[:,'reins':'reins_stopword_count'].values, 
    index=agg_actions.index, 
    columns=total_columns
))

qgrid.show_grid(agg_actions[['year_month', 'editor_id'] + total_columns])

In [ ]:
# Convert to datetime
agg_actions['year_month'] = pd.to_datetime(agg_actions['year_month'])

# Group the data by year month and page (drop the editor information)
agg_actions.drop('editor_id', axis=1).groupby(['year_month','page_id']).sum()

# Listener
from visualization.actions_listener import ActionsListener
listener = ActionsListener(agg_actions)
action_types = (agg_actions.loc[:,'total':'total_stopword_count'].columns.append(
    agg_actions.loc[:,'adds':'reins_stopword_count'].columns)).values.tolist()

# Visualization
from utils.notebooks import get_date_slider_from_datetime
from ipywidgets import interact
from ipywidgets.widgets import Dropdown

interact(listener.listen,
         _range = get_date_slider_from_datetime(agg_actions['year_month']),
         granularity=Dropdown(options=['Yearly', 'Monthly'], value='Yearly'),
         black=Dropdown(options=action_types, value='total'), 
         red=Dropdown(options= ['None'] + action_types, value='total_surv_48h'),
         green=Dropdown(options= ['None'] + action_types, value='None'), 
         blue=Dropdown(options= ['None'] + action_types, value='None'))

In [ ]:
from IPython.display import display, Markdown as md
display(md("---"))
display(md(f"# B. Meassuring conflict"))
display(md(f'## B.1 Tokens Conflict Score'))
display(md(f"***Page: {the_page['title']}***"))

In [ ]:
from metrics.conflict import ConflictCalculator
from wikiwho_wrapper import WikiWho
from IPython.display import clear_output

# create the api
wikiwho = WikiWho(lng='en')

# call the calculator
calculator = ConflictCalculator(the_page.page_id, wikiwho)
calculator.calculate()
clear_output()

# display the tokens, the difference in seconds and its corresponding conflict score
actions = calculator.actions
actions['time_diff_secs'] = actions['time_diff'].dt.total_seconds()
qgrid.show_grid(actions[[
    'action', 'token', 'token_id', 'rev_id', 
    'editor', 'time_diff_secs', 'conflict']].sort_values('conflict', ascending=False))


In [ ]:
from IPython.display import display, Markdown as md
display(md("---"))
display(md(f'## B.2 Conflicting tokens per page'))
display(md(f"***Page: {the_page['title']}***"))

In [ ]:
#editor_actions = calculator.actions[calculator.actions['editor']==editor_inputname]

actions = calculator.actions

# listener
from visualization.wordcloud_listener import WCListener
listener = WCListener(actions, max_words=150)

# # visualization
from utils.notebooks import get_date_slider_from_datetime
from ipywidgets import interact
from ipywidgets.widgets import Dropdown

interact(listener.listen, 
         _range=get_date_slider_from_datetime(actions['rev_time']),
         action=Dropdown(options=['Both', 'Just Insertions', 'Just Deletions'], value='Both'),
         only_conflict = True,
        max_words=100)

In [ ]:
from IPython.display import display, Markdown as md
display(md("---"))
display(md(f'## B.3 Conflict Score and related metrics'))
display(md(f"***Page: {the_page['title']}***"))

In [ ]:
# Visualization
from visualization.conflicts_listener import ConflictsListener
listener = ConflictsListener(actions)

metrics = ['Conflict Score', 'Conflict Ratio', 'Total Conflicts', 
           'Total Elegible Actions', 'Total Actions', 'Total Time',
          'Time per Elegible Action']

display(md(f'**Page conflict score: {calculator.get_page_conflict_score()}**'))

# Visualization
from utils.notebooks import get_date_slider_from_datetime
from ipywidgets import interact
from ipywidgets.widgets import Dropdown

interact(listener.listen,
         _range = get_date_slider_from_datetime(actions['rev_time']),
         granularity=Dropdown(options=['Yearly', 'Monthly', 'Daily'], value='Monthly'),
         black=Dropdown(options=metrics, value='Conflict Score'),
         red=Dropdown(options= ['None'] + metrics, value='None'))

In [ ]:
from IPython.display import display, Markdown as md
display(md("---"))
display(md(f'## B.4 Conflict Score per Editor'))
display(md(f"***Page: {the_page['title']}***"))

In [ ]:
editors_conflicts = calculator.get_conflict_score_per_editor()
qg_obj = qgrid.show_grid(editors_conflicts)
qg_obj


In [ ]:
from IPython.display import display, Markdown as md
display(md("---"))
display(md(f'# C. Selecting a conflicting editor'))
display(md(f"***Page: {the_page['title']}***"))

In [ ]:
from external.wikipedia import WikipediaDV, WikipediaAPI
from ipywidgets import Output

def select_editor(editor):
    global the_editor
    global editor_inputname
    editor_inputname=editor
    
    wikipedia_dv = WikipediaDV(WikipediaAPI(domain='en.wikipedia.org'))
    try:
        the_editor = wikipedia_dv.get_editor(int(editor_inputname))
    except:
        the_editor = wikipedia_dv.get_editor(editor_inputname[2:])

    with out:
        %store the_editor
        %store agg_actions
        %store editor_inputname
        %store calculator
        
        clear_output()
        display(md("### Current Selection:"))
        
        if 'invalid' in the_editor:
            display(f"The editor {editor_inputname} was not found, try a different editor")
        else:
            # display the data that will be passed to the next notebook
            display(the_editor.to_frame('values'))
            display(md(f"#### Evolution of the Conflict Score of *{the_editor['name']}*"))
            _r = tuple(actions['rev_time'][~actions['rev_time'].isnull()].dt.date.agg(
                ['min', 'max']).values)
            ConflictsListener(actions[actions['editor'] == editor_inputname].copy()).listen(
                _r, 'Daily', 'Conflict Score','None')

            


def on_selection_change(change):

    try:
        select_editor(qg_obj.get_selected_df().iloc[0].name)
    except:
        print('Problem parsing the name. Execute the cell again and try a different editor.')
               
qg_obj = qgrid.show_grid(editors_conflicts)
qg_obj.observe(on_selection_change, names=['_selected_rows'])
display(md("### Select one editor row for the next notebook:"))
display(md('**Recomendation:** select an editor with *many conflicts* and *mid-high conflict score*'))
display(qg_obj)
out = Output()
display(out)

# select an editor that does not contain 0| at the beginning
for ed in editors_conflicts.index:
    if ed[:2] != '0|':
        select_editor(ed)
        break


In [ ]:
from IPython.display import HTML
from utils.notebooks import get_next_notebook, get_previous_notebook
if 'the_editor' not in locals() or (the_editor is None) or (len(the_editor) == 0):
    display(md("*There is no conflicting editor.*"))
    display(HTML(f'<a href="{get_previous_notebook()}" target="_blank">Go back to the previous workbook</a>'))
    
elif 'editcount' in the_editor and the_editor['editcount'] > 0:
    if len(editors_conflicts) > 0:
        display(HTML(f'<a href="{get_next_notebook()}" target="_blank">Go to next workbook</a>'))
    else:
        display(HTML(f'<a href="{get_previous_notebook()}" target="_blank">Go back to the previous workbook</a>'))
else:
    display(md("***The editor seems to be invalid. Either it was not not found, or the editcount is zero.***"))
    display(md("*Select a valid editor to continue to the next notebook.*"))
    display(HTML(f'You can also <a href="{get_previous_notebook()}" target="_blank">go back to the previous workbook</a>'))